# Using Notebook Environments to Interact with SQL
© Explore Data Science Academy

## Learning Objectives

In this train, you will learn:
- How to setup a SQL environment in jupyter notebooks, particularly:
    - Serverless SQL - connecting to a database file (SQLite)
    - Client-server SQL - connecting to a database in a SQL server (MySQL)
- How to run SQL queries in Jupyter notebook cells.
- How to list all available tables in a database.


## Outline

To do this, we'll go through the following: 
- Interfacing with SQL Using Magic Commands;
- Connecting to a SQL server database;
- Connecting to a serverless SQL database;
- Listing the tables in a database; and, 
- Running SQL queries.

## SQL Architechtures

There are generally two common architectures for interacting with SQL databases:
- Client-server SQL - database systems that require separate server processes to function. In such cases, the user sets up a connection (typically with user credentials) to the server to interact with a database within it. The server can be located locally on the user's computer or remotely (i.e. on a cloud computer). Examples of  RDBMS (Relational Database Management Systems) that work this way include Microsoft SQL server, MySQL, PostgreSQL, etc.


- Serverless SQL - these systems allow the user to interact directly with the target SQL database, i.e., the SQL database is a file that gets loaded onto the application and can be read or written to using SQL queries. The specific flavour of serverless SQL we will use in this course is SQLite - a portable, serverless, and self-contained version of SQL. Instead of needing to connect to a SQL server for database access, SQLite loads a database from a single `.db` file, this file can be treated in a fashion similar to that of any other data file (e.g. a CSV) with the exception that we read and write to the file using SQL queries.

![SQL Architechtures](https://raw.githubusercontent.com/Explore-AI/Pictures/master/sql-architectures.png)

In this train, we will explore both setups in the context of jupyter notebooks. Before we discuss these setups though, let's first discuss magic functions.

## Interfacing with SQL Using Magic Commands

["Magic" commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) are special commands that are used to enhance the behaviour of IPython. In order to get our notebook to process SQL within each of our cells, we make use of IPython "magic" commands specifically for SQL. These are provided by the [ipython-sql](https://pypi.org/project/ipython-sql/) library. 

To enable these magic commands, we will give a once-off initialisation command: `%load_ext sql`

Next, let's discuss the dependencies for each type of SQL database setup.

## 1. Connecting to a SQL server database 

To connect to a MySQL server database and use our jupyter notebook as a client application, we first need to install some dependencies. This includes the following Python modules: 
 - Running ***locally***: Once-off installations
      - `pip install sqlalchemy`
      - `pip install ipython-sql`
      - `pip install pymysql`

 - Running on ***Google Colab***: For each notebook 
      - `pip install pymysql`

Assuming, you have a local SQL server installed or have access to credentials to a remote SQL server (e.g. AWS RDS), you can connect to it using a connection string of the following form:

`%sql mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}`

In practice, you will need to replace the elements of the database url in the connection string with your given credentials, i.e.:
```
USER - username
PASSWORD - password
HOST - ip address or hostname
PORT - server port number
DATABASE - name of database
```

We will explore how to connect to a SQL server from a jupyter notebook in detail in later trains. 

## 2. Connecting to a serverless SQL database

We can connect to a SQLite database as follows: 

1. We need to install some dependencies. This includes the following Python modules: 

 - Running ***locally***: Once-off installations
      - `pip install sqlalchemy`
      - `pip install ipython-sql`
      - `pip install pymysql`

 - Running on ***Google Colab***: 
      - no dependencies required
      

2. Next, we need a SQLite database file (i.e. with a `.db` file extension) to connect to. For this purpose, the `chinook.db` file has been provided, download it from Athena and place it in the same folder as this notebook or some other known location.

After installing the dependencies and downloading the database file, we can now connect to our DB.

To do this, we simply call an inline magic command `%sql` followed by a [SQLAlchemy-formatted](https://docs.sqlalchemy.org/en/13/core/engines.html#sqlite) DB connection string, i.e.: 

---
`%sql sqlite:////path_to_database_file.db`

    Valid SQLite URL forms are:
     sqlite:///:memory: (or, sqlite://)
     sqlite:///relative/path/to/file.db
     sqlite:////absolute/path/to/file.db

Let's put this into practice by connecting to and querying a SQLite database.

## Listing Database Tables
Before we start we need to load our SQL magic commands (we only need do this once per notebook):

In [1]:
%load_ext sql

If the `chinook.db` file is saved in the same folder as this notebook, then it can be loaded into the notebook as follows:

In [2]:
# Load SQLite database
%sql sqlite:///TMDB.db

If your connection is successful, then you should be able to interact with the DB using standard SQL syntax (provided that you prepend every SQL jupyter cell with `%%sql`).

Before we can run SQL queries, we need to understand what tables exist in the database. This can be achieved by using the following SQL query.

*Note: This query will make more sense as you learn more about SQL queries throughout the course. Your takeaway here is that we had to prepend a jupyter notebook cell with `%%sql` in order to run a SQL query.*

In [3]:
%%sql
SELECT name FROM sqlite_master WHERE type IN ('table','view') AND name NOT LIKE 'sqlite_%' ORDER BY 1

 * sqlite:///TMDB.db
Done.


name
actors
casts
genremap
genres
keywordmap
keywords
languagemap
languages
movies
oscars


In [7]:
%%sql
SELECT * FROM movies
LIMIT 1

 * sqlite:///TMDB.db
Done.


movie_id,title,release_date,budget,homepage,original_language,original_title,overview,popularity,revenue,runtime,release_status,tagline,vote_average,vote_count
5,Four Rooms,1995-12-09 00:00:00.000000,4000000,None,en,Four Rooms,It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.,22.87623,4300000.0,98.0,Released,"Twelve outrageous guests. Four scandalous requests. And one lone bellhop, in his first day on the job, who's in for the wildest New year's Eve of his life.",6.5,530


In [9]:
%%sql
-- Budget of movie inception
SELECT budget
FROM movies
WHERE lower(title) = ('inception')

 * sqlite:///TMDB.db
Done.


budget
160000000


In [10]:
%%sql
--What is the runtime of the movie "Titanic"?
SELECT runtime
FROM movies
WHERE lower(title) = 'titanic'

 * sqlite:///TMDB.db
Done.


runtime
194.0


In [15]:
%%sql
--How many movies do not have English as their original language? (Hint: “en” is the abbreviation for English)
SELECT COUNT(*)
FROM movies
WHERE lower(original_language) NOT LIKE '%en%' 

 * sqlite:///TMDB.db
Done.


COUNT(*)
298


In [17]:
%%sql
--How many movies are there that have a popularity score of more than 250?
SELECT COUNT(*)
FROM movies
WHERE popularity > 250

 * sqlite:///TMDB.db
Done.


COUNT(*)
7


In [25]:
%%sql
--How many movies are there where the title is not the same as the original title?
SELECT COUNT (DISTINCT movie_id)
FROM movies
WHERE lower(title) != lower(original_title) AND original_title NOT LIKE '%?%'
LIMIT 5

 * sqlite:///TMDB.db
Done.


COUNT (DISTINCT movie_id)
175


In [26]:
%%sql
--How many movies are there that managed to get a popularity rating of more than 100 with a budget of less than $10 000 000?
SELECT COUNT(*)
FROM movies
WHERE popularity > 100 AND budget < 10000000

 * sqlite:///TMDB.db
Done.


COUNT(*)
5


In [27]:
%%sql
--How many movies are there that have the word ‘love’ anywhere in the title? (Hint: The L in the word love can be upper or 
--lower case and can be included in words such as ‘lovers’.)
SELECT COUNT(movie_id)
FROM movies
WHERE lower(title) LIKE '%love%'

 * sqlite:///TMDB.db
Done.


COUNT(movie_id)
71


In [29]:
%%sql
--How many movies were released between the dates 1 August 2012 and 31 July 2013?
SELECT COUNT(*)
FROM movies
WHERE release_date BETWEEN '2012-08-01' AND '2013-07-31'

 * sqlite:///TMDB.db
Done.


COUNT(*)
227


In [34]:
%%sql
--You have had a long day and want to sit back and enjoy a movie. Unfortunately, today you are only in
--the mood for a very specific type of movie. It definitely needs to be in English. It should also be new, 
--something after 1 Jan 2010, but not too new as you might have seen it recently, so it must have been 
--released before 1 Jan 2016. It should also be a romantic movie, so make sure it has the word love somewhere in the title. 
--You also want it to be a big blockbuster, so the budget of the movie must be more than $10 000 000.

--What is the movie with the highest popularity that meets all of your requirements?
SELECT movie_id, title, original_title, budget, popularity
FROM movies 
WHERE lower(original_language) LIKE '%en%' AND release_date BETWEEN '2010-01-01' AND '2016-01-01' AND lower(title) LIKE '%love%' AND budget > 10000000
ORDER BY popularity DESC
LIMIT 3

 * sqlite:///TMDB.db
Done.


movie_id,title,original_title,budget,popularity
50646,"Crazy, Stupid, Love.","Crazy, Stupid, Love.",50000000,37.990705
38167,Eat Pray Love,Eat Pray Love,60000000,28.800112
26389,From Paris with Love,From Paris with Love,52000000,27.916284


As you can see, this query outputs all the columns and rows of data that exist in the customers table. 

These "SQL cells" (i.e. cells with `%%sql`) come with all the standard benefits of normal notebook cells, including:
 
 - **Statefullness**: Connection context is maintained between cells, just like standard notebook cells.  
 - **Pretty-printed results:** When runnning in Jupyter, the results of SQL queries are printed in neat tabular results (similar to Pandas formatting). 
 - **Hybrid Python+SQL statements:** Python and SQL code can be used interchangeably within a cell. For more info see [here](https://pypi.org/project/ipython-sql/) 


## Conclusion

In this train, we covered how to setup a SQL database in a jupyter notebook environment and how to run basic queries to show the information stored in the tables of the given database.

## Appendix

The following links may be useful for further learning around the content we've covered: 

 - [SQLite vs. Client-server databases - Trade-off comparison](https://www.sqlite.org/whentouse.html)
 - [IPython-sql reference guide](https://pypi.org/project/ipython-sql/)